### Motivation

Notebook dedicated to demonstrating how to use performance metrics to evaluate various trained models.

### Imports

**Adaptation to recognize the project root. For demonstration purposes only.**

In [1]:
import sys
import os

# Adiciona o diretório raiz ao sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))

**Required**

In [2]:
from ThreeWToolkit.metrics import (
    accuracy_score,
    balanced_accuracy_score
)

### How to

#### Use metrics for classification tasks

**Accuracy Score**

Basic usage

In [3]:
y_true = [1, 0, 1, 1]
y_pred = [1, 0, 0, 1]

acc = accuracy_score(y_true = y_true,
                     y_pred = y_pred)

print(f"The accuracy is {(acc * 100):.3}%.")

The accuracy is 75.0%.


Using sample weight

In [4]:
y_true = [1, 0, 1, 1]
y_pred = [1, 0, 0, 1]
sample_weight = [0.5, 0.2, 0.1, 0.2]

acc = accuracy_score(y_true = y_true,
                     y_pred = y_pred,
                     sample_weight = sample_weight)

print(f"The accuracy is {(acc * 100):.3}%.")

The accuracy is 90.0%.


_________

**Balanced Accuracy Score**

Basic usage

In [5]:
y_true = [1, 0, 1, 1]
y_pred = [1, 0, 0, 1]

balanced_acc = balanced_accuracy_score(y_true = y_true,
                                       y_pred = y_pred)

print(f"The balanced accuracy is {(balanced_acc * 100):.3}%.")

The balanced accuracy is 83.3%.


Using sample weight

In [6]:
y_true = [1, 0, 1, 1]
y_pred = [1, 0, 0, 1]
sample_weight = [1.0, 1.0, 2.0, 1.0]

balanced_acc = balanced_accuracy_score(y_true = y_true,
                                       y_pred = y_pred,
                                       sample_weight = sample_weight)

print(f"The balanced accuracy is {(balanced_acc * 100):.3}%.")

The balanced accuracy is 75.0%.


______